In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.float_format = '${:,.2f}'.format
%matplotlib inline

In [75]:
dc_prop_file ="dataset/DC_Properties.csv"
 
df_prop = pd.read_csv(dc_prop_file)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
df_prop.columns

Index(['Unnamed: 0', 'BATHRM', 'HF_BATHRM', 'HEAT', 'AC', 'NUM_UNITS', 'ROOMS',
       'BEDRM', 'AYB', 'YR_RMDL', 'EYB', 'STORIES', 'SALEDATE', 'PRICE',
       'QUALIFIED', 'SALE_NUM', 'GBA', 'BLDG_NUM', 'STYLE', 'STRUCT', 'GRADE',
       'CNDTN', 'EXTWALL', 'ROOF', 'INTWALL', 'KITCHENS', 'FIREPLACES',
       'USECODE', 'LANDAREA', 'GIS_LAST_MOD_DTTM', 'SOURCE', 'CMPLX_NUM',
       'LIVING_GBA', 'FULLADDRESS', 'CITY', 'STATE', 'ZIPCODE', 'NATIONALGRID',
       'LATITUDE', 'LONGITUDE', 'ASSESSMENT_NBHD', 'ASSESSMENT_SUBNBHD',
       'CENSUS_TRACT', 'CENSUS_BLOCK', 'WARD', 'SQUARE', 'X', 'Y', 'QUADRANT'],
      dtype='object')

In [77]:
df_prop.dtypes

Unnamed: 0              int64
BATHRM                  int64
HF_BATHRM               int64
HEAT                   object
AC                     object
NUM_UNITS             float64
ROOMS                   int64
BEDRM                   int64
AYB                   float64
YR_RMDL               float64
EYB                     int64
STORIES               float64
SALEDATE               object
PRICE                 float64
QUALIFIED              object
SALE_NUM                int64
GBA                   float64
BLDG_NUM                int64
STYLE                  object
STRUCT                 object
GRADE                  object
CNDTN                  object
EXTWALL                object
ROOF                   object
INTWALL                object
KITCHENS              float64
FIREPLACES              int64
USECODE                 int64
LANDAREA                int64
GIS_LAST_MOD_DTTM      object
SOURCE                 object
CMPLX_NUM             float64
LIVING_GBA            float64
FULLADDRES

In [172]:
#Highest storied building address
df_prop['STORIES'] = df_prop['STORIES'].astype(np.int16)
df_prop['STORIES'] = df_prop['STORIES'].fillna(0)
df_prop['STORIES'].unique()
df_prop.loc[df_prop['STORIES'] >100,['FULLADDRESS','CITY','SALEDATE','PRICE','STORIES']]

,FULLADDRESS,CITY,SALEDATE,PRICE,STORIES
7546,664 E STREET NE,WASHINGTON,2015-01-26 00:00:00,"$760,000.00",826
20491,2712 N STREET NW,WASHINGTON,2015-09-23 00:00:00,"$1,459,000.00",250
21170,2912 N STREET NW,WASHINGTON,2011-02-25 00:00:00,"$5,397,500.00",275
21930,2721 O STREET NW,WASHINGTON,2017-01-27 00:00:00,"$1,725,000.00",275


In [81]:
#Mean price of houses by year -- include house address, city, state, zipcode
#df_prop[df_prop['PRICE'].isnull()] = df_prop['PRICE'].mean() #Set NaN values to mean price

df_prop['EYB'] = df_prop['EYB'].astype("int")

df_mean_by_year = df_prop.groupby(['EYB'])['PRICE'].mean()
df_mean_by_year.head()

EYB
1800   $233,000.00
1820   $447,500.00
1865   $402,250.00
1870   $738,156.25
1875   $482,308.27
Name: PRICE, dtype: float64

In [82]:
df_prop.drop_duplicates(inplace=True)

In [83]:
#ZIPCODE clean up 

df_prop['ZIPCODE'].unique()
df_prop.astype({'ZIPCODE':str}).head()
df_prop['ZIPCODE'].dtype

dtype('float64')

In [84]:
df_prop['ZIPCODE'] = df_prop['ZIPCODE'].astype(str)
df_prop['ZIPCODE'] = df_prop['ZIPCODE'].str.slice(0,5)

In [85]:
df_prop['ZIPCODE'].unique()

array(['20009', '20037', '20036', '20005', '20052', '20008', '20001',
       '20024', '20006', '20003', '20002', '20007', '20016', '20015',
       '20011', '20010', '20012', '20017', '20018', '20019', '20020',
       '20032', 'nan', '20392', '20004'], dtype=object)

In [205]:
df_prop['ZIPCODE'] = df_prop['ZIPCODE'].astype('int')
propgb = df_prop.groupby('ZIPCODE')

In [206]:
propgb.get_group(20016)['PRICE'].describe() #for a given group describe the distribution

count        $10,644.00
mean      $3,591,286.14
std      $20,426,251.06
min               $0.00
25%               $0.00
50%         $285,000.00
75%         $784,625.00
max     $137,427,545.00
Name: PRICE, dtype: float64

In [237]:
def mean_price_by_zip(p_by_gb, z):
    return p_by_gb.get_group(z)['PRICE'].mean()
mean_price_by_zip(propgb,20016)
#Fix zero price rows to mean price by zipcode
#df_prop[df_prop['PRICE']==0].apply(mean_price_by_zip(propgb,df_prop['ZIPCODE'])) # THIS DID NOT WORK

3591286.137636227

In [239]:
#pivot table
df_prop.pivot_table(index='ZIPCODE',values='PRICE')

,PRICE
ZIPCODE,
0,$0.00
20001,"$380,499.33"
20002,"$301,601.84"
20003,"$391,474.06"
20004,"$393,378.20"
20005,"$2,255,303.23"
20006,"$114,555.73"
20007,"$595,084.93"
20008,"$518,559.09"
